In [101]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import numpy as np
import evaluate
import torch

In [102]:
tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en",model_max_length=128)
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en")

loading file spiece.model from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\spiece.model
loading file tokenizer.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\mtx/.cache\huggingface\hub\models--KETI-AIR-Downstream--long-ke-t5-base-translation-aihub-ko2en\snapshots\ec267d5ac800975de819e962a1fb2cb0897f277a\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\mtx/.cac

In [103]:
def prepare_dataset(data):
    source_language = [value['ko'] for key, value in data.items()]
    target_language = [value['en'] for key, value in data.items()]
    return source_language, target_language

In [104]:
train = load_dataset("iwslt2017","iwslt2017-ko-en", split="train")

In [105]:
train[0]

{'translation': {'en': 'Thank you so much, Chris.',
  'ko': '감사합니다, 크리스. 이곳에 두 번이나'}}

In [106]:
source_lang = "ko"
target_lang = "en"
prefix = "translate English to Korean: "


def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [107]:
tokenized_train = train.map(preprocess_function, batched=True)

Map:   0%|          | 0/230240 [00:00<?, ? examples/s]

In [108]:
tokenized_train[0]

{'translation': {'en': 'Thank you so much, Chris.',
  'ko': '감사합니다, 크리스. 이곳에 두 번이나'},
 'input_ids': [1092,
  5778,
  513,
  7,
  20006,
  20004,
  20153,
  5850,
  20005,
  27,
  981,
  8,
  135,
  361,
  188,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [21257, 20025, 20078, 20182, 20006, 23522, 20005, 1]}

In [89]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [90]:
from transformers import TrainingArguments, Trainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
    remove_unused_columns=False,
    logging_dir="./logs",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


import transformers
transformers.logging.set_verbosity_info()

trainer.train()

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 1
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1
  Number of trainable parameters = 296,696,448


AttributeError: 'list' object has no attribute 'keys'